In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [2]:
%cd chatGLM-6B-QLoRA
!git pull --all --force 
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 122.77 KiB | 3.07 MiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   3d73fa8..4c06287  main       -> origin/main
Updating 3d73fa8..4c06287
Fast-forward
 chatglm-rewardmodel-qlora.ipynb                    |    2 +-
 rm_3.py                                            | 1602 ++++++++++++++++++++
 ...50\200\203\344\270\216\347\273\217\351\252\214" |   23 +
 3 files changed, 1626 insertions(+), 1 deletion(-)
 create mode 100644 rm_3.py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# 

In [ ]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0 python  rm_trl.py \
--model_name 'THUDM/chatglm2-6b' \
--resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/reward_model_0809_v1/checkpoint-50 \
--num_train_epochs 2 \
--gradient_accumulation_steps 1 \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size  4 \
--max_length 512 \
--output_dir ./reward_model_0810_v2 \
--train_subset 80 \
--eval_subset 20 \
--local_rank 0  \
--bf16 False

# 上面的warning There were missing keys in the checkpoint model loaded: 'transformer.embedding.word_embeddings.weight', 'transformer.rotary_pos_emb.inv_freq', 'transformer.encoder.layers.0.input_layernorm.weight', ....
# 是 trainingArguments中设置 load_best_model_at_end = True 后出现的
# 说明 trainer的确在最后把最好的那个adapters的参数做了load（只不过load时strict=True要求严格匹配了）
# 这说明保存在output_dir中的pytorch_model.bin和vhead.bin参数都是最佳的 我们来验证下

验证成功  跑的是py中的test_load_best()


# deepspeed 单机多卡从头训练 reward model

In [8]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json


Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 1.80 KiB | 461.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   91e90a0..7a781c3  main       -> origin/main
Updating 91e90a0..7a781c3
Fast-forward
 ...2\350\200\203\344\270\216\347\273\217\351\252\214" | 19 +++++++++++++++++++
 1 file changed, 19 insertions(+)
[2023-08-11 12:40:30,370] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load lib

--deepspeed ds_zero2_config.json

# 单机多卡deepspeed 从ckpt继续训练 

In [13]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v3 \
  --resume_from_checkpoint output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 696 bytes | 232.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   52b2f25..3d73fa8  main       -> origin/main
Updating 52b2f25..3d73fa8
Fast-forward
 rewardmodel_qlora_chatglm2.py | 5 +----
 1 file changed, 1 insertion(+), 4 deletions(-)
[2023-08-11 14:16:14,392] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so

In [14]:
ls -lrt /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0811-v3

total 463388
-rw-r--r-- 1 root root      9014 Aug 11 13:49 events.out.tfevents.1691760494.5d48eec2301b.662.0
drwxr-xr-x 2 root root      4096 Aug 11 14:29 checkpoint-20/
drwxr-xr-x 2 root root      4096 Aug 11 14:42 checkpoint-50/
-rw-r--r-- 1 root root      9014 Aug 11 14:42 events.out.tfevents.1691763633.5d48eec2301b.814.0
-rw-r--r-- 1 root root     17140 Aug 11 14:42 value_head.bin
-rw-r--r-- 1 root root      1751 Aug 11 14:42 config.json
-rw-r--r-- 1 root root 474446225 Aug 11 14:42 pytorch_model.bin


# 单卡训练 reward model

In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  10  \
  --save_total_limit 2 \
  --load_in_4bit True 


In [17]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0812-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

Fetching origin
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 1.35 KiB | 197.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   865a610..1cdf9eb  main       -> origin/main
Updating 865a610..1cdf9eb
Fast-forward
 rm_3.py | 12 +++++++-----
 1 file changed, 7 insertions(+), 5 deletions(-)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/lib

In [16]:
rm -rf output-rm-1k-0812-v1

参考 https://github.com/hiyouga/ChatGLM-Efficient-Tuning/blob/main/src/glmtuner/tuner/core/adapter.py#L83
实现 resume——from --checkpoint

参考

In [ ]:
https://huggingface.co/docs/trl/models#trl.AutoModelForCausalLMWithValueHead
    https://github.com/yongzhuo/chatglm-maths/blob/main/chatglm_maths/t10_lora_trl_train_ppo.py#L175
        https://huggingface.co/docs/trl/v0.5.0/en/customization#use-the-cuda-cache-optimizer
            https://huggingface.co/docs/trl/trainer#trl.RewardTrainer